In [21]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.signal import welch
import os

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    遍历 input_folder 下的所有 CSV 文件，处理 EMG 和 IMU 数据，并保存特征矩阵和标签到 output_folder。
    """
    # 设定采样率
    fs = 1000  # 采样率 Hz
    cycle_duration = 10  # 每个周期 10 秒
    skip_seconds = 5  # 跳过前 4 秒
    use_seconds = 5  # 需要保留的秒数

    cycle_samples = fs * cycle_duration  # 10 秒数据点数 = 10000
    skip_samples = fs * skip_seconds  # 跳过 4 秒 = 4000
    use_samples = fs * use_seconds  # 取后 6 秒 = 6000

    # 滑动窗口参数
    window_size = 200  # 200ms = 200 采样点
    step_size = 100  # 100ms = 100 采样点
    num_windows = (use_samples - window_size) // step_size + 1  # 计算窗口数

    # 创建存储文件夹
    os.makedirs(output_folder, exist_ok=True)

    # **读取 shuffle_order.xlsx**
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    print(shuffle_df.shape)
    
    if shuffle_df.shape[0] < 15:
        raise ValueError("标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。")

    # 存储所有数据和标签
    all_features = []
    all_labels = []

    # **遍历 CSV 文件**
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx+1}/{len(csv_files)})")

        # **读取数据**
        raw_data = pd.read_csv(file_path)

        # **检查通道数（忽略时间列）**
        num_channels = raw_data.shape[1] - 1
        print(f"Detected {num_channels} channels (excluding time column).")

        # **归一化（Z-score 标准化）**
        data = raw_data.iloc[:, 1:]  # 去掉时间列
        mean_vals = data.mean(axis=0)
        std_vals = data.std(axis=0)
        normalized_data = (data - mean_vals) / (std_vals + 1e-10)  # 避免除零错误

        # **更新原始数据**
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        segments = []  # 存储所有分割后的数据
        labels = shuffle_df.iloc[file_idx].values.tolist()  # 获取该文件对应的 26 组标签
        num_cycles = 26  # 每个文件固定 26 段（每段 10s）

        for i in range(num_cycles):
            start_idx = i * cycle_samples + skip_samples  # 跳过前 4 秒
            end_idx = start_idx + use_samples  # 取后 6 秒

            if end_idx > len(raw_data):  # 处理不足 6000 采样点的情况
                segment = raw_data.iloc[start_idx:].values  # 取剩余数据
                pad_size = use_samples - len(segment)  # 计算填充数
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)  # 填充 0
            else:
                segment = raw_data.iloc[start_idx:end_idx].values  # 正常提取数据

            # **滑动窗口**
            windows = [
                segment[j:j + window_size, 1:]  # 取 200 采样点，忽略时间列
                for j in range(0, use_samples - window_size + 1, step_size)  # 滑动步长 100
            ]
            segments.append(np.array(windows))

        # **转换为 NumPy 数组**
        segments_array = np.array(segments)  # 形状 (26, num_windows, 200, num_channels)

        # **计算特征**
        features_batches = []
        for batch_idx in range(segments_array.shape[0]):  # 26 个 batch
            batch_features = []  # 存储当前 batch 的所有窗口特征
            for window_idx in range(segments_array.shape[1]):  # 计算每个窗口
                window = segments_array[batch_idx, window_idx]  # (200, num_channels)
                features = extract_features(window)  # 计算 (15, 10)
                batch_features.append(features)  # 存入 batch

            features_batches.append(np.array(batch_features))

        features_array = np.array(features_batches)  # (26, num_windows, 15, 10)

        # **存储数据**
        all_features.append(features_array)
        all_labels.extend(labels)  # 每个文件 26个 cycle，取 shuffle_order.xlsx 里的前 26 个标签

    # **最终转换为 NumPy 数组**
    all_features = np.vstack(all_features)  # 合并所有 batch，形状 (总 batch, num_windows, 15, 10)
    all_labels = np.array(all_labels)  # (总 batch,)

    # **保存**
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")


def extract_features(segment):
    """
    计算 EMG 和 IMU 的 15 个特征，并拼接成 (15, 10)
    """
    # **分离 EMG (前四列) 和 IMU (后六列)**
    emg_signals = segment[:, :4]  # 4 个 EMG 通道 (200, 4)
    imu_signals = segment[:, 4:]  # 6 个 IMU 通道 (200, 6)

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1)  # (15, 10)

# **运行函数**
root = r"data\G"
input_folder = root
output_folder = os.path.join(root, "windowed_data")
shuffle_order_file = os.path.join(root,"shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)


(15, 26)
Processing sensor_data1.csv (1/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -3.775955
1       -3.775955
2       -0.214248
3       -0.214248
4       -0.214248
           ...   
45715   -0.005459
45716   -0.017740
45717   -0.017740
45718   -0.017740
45719   -0.017740
Name: EMG1, Length: 45720, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -6.768217
1       -6.768217
2        0.054180
3        0.076403
4        0.054180
           ...   
45715   -0.012488
45716   -0.012488
45717   -0.034711
45718   -0.056934
45719   -0.034711
Name: EMG2, Length: 45720, dtype: float64' h

Processing sensor_data10.csv (2/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -5.279856
1       -5.279856
2       -0.008469
3       -0.008469
4       -0.008469
           ...   
46195   -0.043035
46196   -0.043035
46197   -0.043035
46198   -0.112168
46199   -0.146735
Name: EMG1, Length: 46200, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -7.967664
1       -7.967664
2        0.052054
3        0.025846
4        0.025846
           ...   
46195   -0.052779
46196   -0.052779
46197   -0.026571
46198   -0.000362
46199   -0.000362
Name: EMG2, Length: 46200, dtype: float64' h

Processing sensor_data11.csv (3/15)
Detected 10 channels (excluding time column).


...

Processing sensor_data8.csv (14/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -4.303589
1       -4.303589
2       -0.032001
3       -0.032001
4       -0.032001
           ...   
45996   -0.088206
45997   -0.088206
45998   -0.088206
45999   -0.102257
46000   -0.088206
Name: EMG1, Length: 46001, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -6.680219
1       -6.680219
2       -0.269486
3       -0.269486
4       -0.269486
           ...   
45996    0.037878
45997    0.015923
45998    0.015923
45999    0.015923
46000   -0.006031
Name: EMG2, Length: 46001, dtype: float64' h

Processing sensor_data9.csv (15/15)
Detected 10 channels (excluding time column).


C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -4.303005
1       -4.303005
2        0.506601
3        0.562690
4        0.604756
           ...   
46333   -0.096353
46334   -0.110375
46335   -0.096353
46336   -0.124397
46337   -0.096353
Name: EMG1, Length: 46338, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_10696\1407476850.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -7.636328
1       -7.636328
2       -0.081067
3       -0.030866
4       -0.005765
           ...   
46333   -0.005765
46334    0.019335
46335    0.019335
46336    0.044436
46337    0.044436
Name: EMG2, Length: 46338, dtype: float64' h

Feature extraction complete! Shape: (390, 49, 15, 10)
Labels saved: (390,)
Feature matrix saved at: data\G\windowed_data/feature_matrix.npy
Labels saved at: data\G\windowed_data/labels.npy


In [15]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.signal import welch
import os

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    遍历 input_folder 下的所有 CSV 文件，处理 EMG 和 IMU 数据，并保存特征矩阵和标签到 output_folder。
    """
    # 设定采样率
    original_fs = 1000  # 原采样率 Hz
    target_fs = 200  # 目标采样率 Hz
    downsample_factor = original_fs // target_fs  # 降采样因子
    cycle_duration = 10  # 每个周期 10 秒
    skip_seconds = 5  # 跳过前 4 秒
    use_seconds = 5  # 需要保留的秒数

    cycle_samples = (original_fs * cycle_duration) // downsample_factor  # 2000
    skip_samples = (original_fs * skip_seconds) // downsample_factor  # 1000
    use_samples = (original_fs * use_seconds) // downsample_factor  # 1000

    # 滑动窗口参数
    window_size = 100  # 200ms = 40 采样点 (原 200, 降采样后 40)
    step_size = 50  # 100ms = 20 采样点 (原 100, 降采样后 20)
    num_windows = (use_samples - window_size) // step_size + 1  # 计算窗口数

    # 创建存储文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 读取 shuffle_order.xlsx
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    if shuffle_df.shape[0] < 15:
        raise ValueError("标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。")

    all_features = []
    all_labels = []
    
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx+1}/{len(csv_files)})")

        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1
        print(f"Detected {num_channels} channels (excluding time column).")

        # 降采样
        raw_data = raw_data.iloc[::downsample_factor, :].reset_index(drop=True)
        
        # 归一化
        data = raw_data.iloc[:, 1:]
        mean_vals = data.mean(axis=0)
        std_vals = data.std(axis=0)
        normalized_data = (data - mean_vals) / (std_vals + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        segments = []
        labels = shuffle_df.iloc[file_idx].values.tolist()
        num_cycles = 26

        for i in range(num_cycles):
            start_idx = i * cycle_samples + skip_samples
            end_idx = start_idx + use_samples

            if end_idx > len(raw_data):
                segment = raw_data.iloc[start_idx:].values
                pad_size = use_samples - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            else:
                segment = raw_data.iloc[start_idx:end_idx].values

            windows = [
                segment[j:j + window_size, 1:]
                for j in range(0, use_samples - window_size + 1, step_size)
            ]
            segments.append(np.array(windows))

        segments_array = np.array(segments)
        
        features_batches = []
        for batch_idx in range(segments_array.shape[0]):
            batch_features = []
            for window_idx in range(segments_array.shape[1]):
                window = segments_array[batch_idx, window_idx]
                features = extract_features(window)
                batch_features.append(features)

            features_batches.append(np.array(batch_features))

        features_array = np.array(features_batches)

        all_features.append(features_array)
        all_labels.extend(labels)

    all_features = np.vstack(all_features)
    all_labels = np.array(all_labels)

    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")

def extract_features(segment):
    emg_signals = segment[:, :4]
    imu_signals = segment[:, 4:]

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=200, nperseg=40)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=200, nperseg=40)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T

    return np.concatenate((emg_features, imu_features), axis=1)

root = r"data\G"
input_folder = root
output_folder = os.path.join(root, "windowed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)

ValueError: 标签文件数据不足 15 组！请检查 `shuffle_order.xlsx`。

In [5]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\G\windowed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



Loaded features from E:\MSC\Spring\AML\GestureLink\data\G\windowed_data\feature_matrix.npy, shape: (390, 19, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\G\windowed_data\labels.npy, shape: (390,)
Training set: X_train: (312, 19, 15, 10), y_train: (312,)
Testing set: X_test: (78, 19, 15, 10), y_test: (78,)


---


# 处理分开的数据


In [ ]:
import numpy as np
import pandas as pd
import os
import scipy.stats as stats
from scipy.signal import welch

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    处理 EMG 数据文件，先按照 shuffle_order.xlsx 划分手势数据，
    然后进行窗口切分，并提取特征。
    """
    fs = 1000  # 采样率 1000Hz，每秒 1000 个数据点
    target_samples = 5000  # 每个手势的固定采样点数
    window_size = 200  # 窗口大小 400 采样点（0.4s）
    step_size = 100  # 窗口滑动步长 200 采样点（0.2s）

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 读取 shuffle_order.xlsx
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    if shuffle_df.shape[1] < 26:
        raise ValueError("标签文件数据不足 26 组！请检查 `shuffle_order.xlsx`。")

    all_features = []
    all_labels = []
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx + 1}/{len(csv_files)})")

        # 读取数据
        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1  # 去掉时间列
        print(f"Detected {num_channels} channels (excluding time column).")

        # 归一化数据
        data = raw_data.iloc[:, 1:]
        normalized_data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        # 按 shuffle_order.xlsx 划分 26 组手势
        valid_segments = []
        for i in range(26):
            start_idx = shuffle_df.iloc[file_idx, i]  # 该手势起始索引
            if start_idx + target_samples <= len(raw_data):
                segment = raw_data.iloc[start_idx:start_idx + target_samples, 1:].values
            else:
                segment = raw_data.iloc[start_idx:, 1:].values
                pad_size = target_samples - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            valid_segments.append(segment)

        valid_segments = np.array(valid_segments)  # 形状 (26, 5000, num_channels)

        # 进行窗口划分和特征提取
        features_batches = []
        for seg in valid_segments:
            windows = [seg[j:j + window_size, :] for j in range(0, target_samples - window_size + 1, step_size)]
            batch_features = np.array([extract_features(window) for window in windows])  # (num_windows, 15, num_channels)
            features_batches.append(batch_features)

        processed_segments = np.array(features_batches)  # (26, num_windows, 15, num_channels)

        if processed_segments.shape[0] != 26:
            print(f"Warning: {file_name} 提取到 {processed_segments.shape[0]} 个手势，不是 26 个。")

        # 存储数据
        all_features.append(processed_segments)
        all_labels.extend(shuffle_df.iloc[file_idx, :26].values.tolist())

    # 最终转换为 NumPy 数组
    all_features = np.vstack(all_features)  # (总手势数, num_windows, 15, num_channels)
    all_labels = np.array(all_labels)  # (总手势数,)

    # 保存数据
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")

def extract_features(segment):
    """
    计算 EMG 和 IMU 的特征。
    """
    emg_signals = segment[:, :4]  # 4 个 EMG 通道
    imu_signals = segment[:, 4:]  # 6 个 IMU 通道

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1)  # (15, 10)

# 运行代码
root = r"data\FZH"
input_folder = root
output_folder = os.path.join(root, "processed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)

In [12]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



Loaded features from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\feature_matrix.npy, shape: (390, 49, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\FZH\processed_data\labels.npy, shape: (390,)
Training set: X_train: (312, 49, 15, 10), y_train: (312,)
Testing set: X_test: (78, 49, 15, 10), y_test: (78,)


In [ ]:
import numpy as np
import pandas as pd
import os
import scipy.stats as stats
from scipy.signal import welch

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    处理 EMG 数据文件，先按照 shuffle_order.xlsx 划分手势数据，
    然后进行窗口切分，并提取特征。
    """
    fs = 1000  # 采样率 1000Hz，每秒 1000 个数据点
    target_samples = 5000  # 每个手势的固定采样点数
    window_size = 200  # 窗口大小 400 采样点（0.4s）
    step_size = 100  # 窗口滑动步长 200 采样点（0.2s）

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 读取 shuffle_order.xlsx
    shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
    if shuffle_df.shape[1] < 26:
        raise ValueError("标签文件数据不足 26 组！请检查 `shuffle_order.xlsx`。")

    all_features = []
    all_labels = []
    csv_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".csv")])

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name} ({file_idx + 1}/{len(csv_files)})")

        # 读取数据
        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1  # 去掉时间列
        print(f"Detected {num_channels} channels (excluding time column).")

        # 归一化数据
        data = raw_data.iloc[:, 1:]
        normalized_data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        # 按 shuffle_order.xlsx 划分 26 组手势
        valid_segments = []
        for i in range(26):
            start_idx = shuffle_df.iloc[file_idx, i]  # 该手势起始索引
            if start_idx + target_samples <= len(raw_data):
                segment = raw_data.iloc[start_idx:start_idx + target_samples, 1:].values
            else:
                segment = raw_data.iloc[start_idx:, 1:].values
                pad_size = target_samples - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            valid_segments.append(segment)

        valid_segments = np.array(valid_segments)  # 形状 (26, 5000, num_channels)

        # 进行窗口划分和特征提取
        features_batches = []
        for seg in valid_segments:
            windows = [seg[j:j + window_size, :] for j in range(0, target_samples - window_size + 1, step_size)]
            batch_features = np.array([extract_features(window) for window in windows])  # (num_windows, 15, num_channels)
            features_batches.append(batch_features)

        processed_segments = np.array(features_batches)  # (26, num_windows, 15, num_channels)

        if processed_segments.shape[0] != 26:
            print(f"Warning: {file_name} 提取到 {processed_segments.shape[0]} 个手势，不是 26 个。")

        # 存储数据
        all_features.append(processed_segments)
        all_labels.extend(shuffle_df.iloc[file_idx, :26].values.tolist())

    # 最终转换为 NumPy 数组
    all_features = np.vstack(all_features)  # (总手势数, num_windows, 15, num_channels)
    all_labels = np.array(all_labels)  # (总手势数,)

    # 保存数据
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)

    print(f"Feature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")
    print(f"Feature matrix saved at: {output_folder}/feature_matrix.npy")
    print(f"Labels saved at: {output_folder}/labels.npy")

def extract_features(segment):
    """
    计算 EMG 和 IMU 的特征。
    """
    emg_signals = segment[:, :4]  # 4 个 EMG 通道
    imu_signals = segment[:, 4:]  # 6 个 IMU 通道

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ])

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ])

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1)  # (15, 10)

# 运行代码
root = r"data\FZH"
input_folder = root
output_folder = os.path.join(root, "processed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")
process_emg_folder(input_folder, output_folder, shuffle_order_file)

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\WXR\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



In [3]:
import numpy as np
import pandas as pd
import os
import re
import scipy.stats as stats
from scipy.signal import welch

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    处理 EMG 数据文件：
    - 通过 **时间戳间隔** 自动识别 26 个手势
    - **不足 5000 采样点的手势补零**
    - **确保最终 shape = (390, 49, 15, 10)，每个手势仅有一个标签**
    """
    fs = 1000  # 采样率 1000Hz
    window_size = 200  # 窗口大小 200 采样点（0.2s）
    step_size = 100  # 滑动步长 100 采样点（0.1s）
    time_gap_threshold = 500  # 手势分界时间间隔（单位：ms）

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # **按正确顺序读取 CSV 文件**
    csv_files = sorted(
        [f for f in os.listdir(input_folder) if f.startswith("sensor_data") and f.endswith(".csv")],
        key=lambda x: int(re.search(r"sensor_data(\d+)\.csv", x).group(1))
    )

    all_features = []
    all_labels = []

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"\nProcessing {file_name} ({file_idx + 1}/{len(csv_files)})")

        # **读取数据**
        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1  # 去掉时间列
        print(f"Detected {num_channels} channels (excluding time column).")

        # **归一化数据**
        data = raw_data.iloc[:, 1:]
        normalized_data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        # **获取时间戳**
        timestamps = raw_data.iloc[:, 0].astype(float).values  # 确保时间戳是 float 类型

        # **检测手势分界点（时间间隔 > 500ms）**
        segment_starts = [0]
        for i in range(1, len(timestamps)):
            if timestamps[i] - timestamps[i - 1] > time_gap_threshold:
                segment_starts.append(i)

        segment_starts = segment_starts[:26]  # 只保留前 26 个手势
        segment_ends = segment_starts[1:] + [len(raw_data)]  # 计算手势结束点

        print(f"\n[INFO] 识别到 {len(segment_starts)} 个手势")
        for i, (start, end) in enumerate(zip(segment_starts, segment_ends)):
            print(f"手势 {i+1}: 开始索引 {start}, 结束索引 {end}, 长度 {end - start}")

        # **切割手势数据**
        valid_segments = []
        for start, end in zip(segment_starts, segment_ends):
            segment = raw_data.iloc[start:end, 1:].values  # 提取特征列

            # **确保手势片段足够长，否则填充 0**
            if len(segment) < 900:
                # print(len(segment))
                pad_size = 900 - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            else:
                segment = segment[:900]  # 截断过长的手势

            valid_segments.append(segment)

        valid_segments = np.array(valid_segments)  # (26, 5000, num_channels)

        # **读取 `shuffle_order.xlsx` 以匹配手势标签**
        shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
        if file_idx >= len(shuffle_df):
            print(f"[WARNING] `shuffle_order.xlsx` 只有 {len(shuffle_df)} 行，但 `file_idx={file_idx}`，跳过该文件。")
            continue
        
        gesture_labels = shuffle_df.iloc[file_idx, :len(valid_segments)].values.tolist()  # 取前 `26` 个标签

        # **窗口化 & 特征提取**
        features_batches = []
        label_list = []
        for idx, seg in enumerate(valid_segments):
            num_samples = seg.shape[0]
            windows = [seg[j:j + window_size, :] for j in range(0, num_samples - window_size + 1, step_size)]
            
            batch_features = np.array([extract_features(window) for window in windows])  # (49, 15, num_channels)
            features_batches.append(batch_features)

            # **仅保留 1 个手势的标签**
            label_list.append(gesture_labels[idx])  

        processed_segments = np.array(features_batches)  # (26, 49, 15, 10)

        # **存储数据**
        all_features.append(processed_segments)
        all_labels.extend(label_list)  # **每个手势只对应 1 个标签**

    # **转换为 NumPy 数组**
    all_features = np.vstack(all_features)  # (390, 49, 15, 10)
    all_labels = np.array(all_labels)  # (390,)

    print(f"\nFeature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")

    # **保存数据**
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)


def extract_features(segment):
    """
    计算 EMG 和 IMU 的特征，返回 `float64` 类型
    """
    emg_signals = segment[:, :4].astype(np.float64)  # 4 个 EMG 通道
    imu_signals = segment[:, 4:].astype(np.float64)  # 6 个 IMU 通道

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ], dtype=np.float64)

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ], dtype=np.float64)

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1).astype(np.float64)  # (15, 10)


# **运行代码**
root = r"data\WXR"
input_folder = root
output_folder = os.path.join(root, "processed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")

process_emg_folder(input_folder, output_folder, shuffle_order_file)



Processing sensor_data1.csv (1/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 835, 长度 835
手势 2: 开始索引 835, 结束索引 1686, 长度 851
手势 3: 开始索引 1686, 结束索引 2535, 长度 849
手势 4: 开始索引 2535, 结束索引 3399, 长度 864
手势 5: 开始索引 3399, 结束索引 4264, 长度 865
手势 6: 开始索引 4264, 结束索引 5135, 长度 871
手势 7: 开始索引 5135, 结束索引 5983, 长度 848
手势 8: 开始索引 5983, 结束索引 6850, 长度 867
手势 9: 开始索引 6850, 结束索引 7692, 长度 842
手势 10: 开始索引 7692, 结束索引 8534, 长度 842
手势 11: 开始索引 8534, 结束索引 9378, 长度 844
手势 12: 开始索引 9378, 结束索引 10222, 长度 844
手势 13: 开始索引 10222, 结束索引 11099, 长度 877
手势 14: 开始索引 11099, 结束索引 11964, 长度 865
手势 15: 开始索引 11964, 结束索引 12837, 长度 873
手势 16: 开始索引 12837, 结束索引 13700, 长度 863
手势 17: 开始索引 13700, 结束索引 14554, 长度 854
手势 18: 开始索引 14554, 结束索引 15418, 长度 864
手势 19: 开始索引 15418, 结束索引 16286, 长度 868
手势 20: 开始索引 16286, 结束索引 17133, 长度 847
手势 21: 开始索引 17133, 结束索引 17995, 长度 862
手势 22: 开始索引 17995, 结束索引 18860, 长度 865
手势 23: 开始索引 18860, 结束索引 19698, 长度 838
手势 24: 开始索引 19698, 结束索引 20554, 长度 856
手势 25: 开始索引 20554, 结束索引 

C:\Users\user\AppData\Local\Temp\ipykernel_26108\334003276.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.104279
1       -0.079371
2       -0.087674
3       -0.104279
4       -0.079371
           ...   
22264   -0.411475
22265   -0.428080
22266   -0.428080
22267   -0.461290
22268   -0.469593
Name: 303, Length: 22269, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\334003276.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.140593
1       -0.097321
2       -0.113057
3       -0.148461
4       -0.124858
           ...   
22264   -0.286144
22265   -0.266475
22266   -0.266475
22267   -0.309747
22268   -0.313680
Name: 306, Length: 22269, dtype: float64' has d


Processing sensor_data2.csv (2/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 828, 长度 828
手势 2: 开始索引 828, 结束索引 1674, 长度 846
手势 3: 开始索引 1674, 结束索引 2520, 长度 846
手势 4: 开始索引 2520, 结束索引 3380, 长度 860
手势 5: 开始索引 3380, 结束索引 4244, 长度 864
手势 6: 开始索引 4244, 结束索引 5096, 长度 852
手势 7: 开始索引 5096, 结束索引 5956, 长度 860
手势 8: 开始索引 5956, 结束索引 6811, 长度 855
手势 9: 开始索引 6811, 结束索引 7659, 长度 848
手势 10: 开始索引 7659, 结束索引 8529, 长度 870
手势 11: 开始索引 8529, 结束索引 9388, 长度 859
手势 12: 开始索引 9388, 结束索引 10254, 长度 866
手势 13: 开始索引 10254, 结束索引 11107, 长度 853
手势 14: 开始索引 11107, 结束索引 11955, 长度 848
手势 15: 开始索引 11955, 结束索引 12814, 长度 859
手势 16: 开始索引 12814, 结束索引 13678, 长度 864
手势 17: 开始索引 13678, 结束索引 14545, 长度 867
手势 18: 开始索引 14545, 结束索引 15396, 长度 851
手势 19: 开始索引 15396, 结束索引 16255, 长度 859
手势 20: 开始索引 16255, 结束索引 17088, 长度 833
手势 21: 开始索引 17088, 结束索引 17941, 长度 853
手势 22: 开始索引 17941, 结束索引 18802, 长度 861
手势 23: 开始索引 18802, 结束索引 19666, 长度 864
手势 24: 开始索引 19666, 结束索引 20516, 长度 850
手势 25: 开始索引 20516, 结束索引 

C:\Users\user\AppData\Local\Temp\ipykernel_26108\334003276.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.390440
1        0.449669
2        0.459540
3        0.508897
4        0.538511
           ...   
22247   -0.191971
22248   -0.182100
22249   -0.211714
22250   -0.182100
22251   -0.172228
Name: 345, Length: 22252, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\334003276.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.008569
1        0.035294
2       -0.037810
3       -0.008569
4        0.049915
           ...   
22247   -0.266870
22248   -0.223007
22249   -0.286364
22250   -0.276617
22251   -0.223007
Name: 333, Length: 22252, dtype: float64' has d


Processing sensor_data3.csv (3/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 817, 长度 817
手势 2: 开始索引 817, 结束索引 1675, 长度 858
手势 3: 开始索引 1675, 结束索引 2527, 长度 852
手势 4: 开始索引 2527, 结束索引 3395, 长度 868
手势 5: 开始索引 3395, 结束索引 4238, 长度 843
手势 6: 开始索引 4238, 结束索引 5077, 长度 839
手势 7: 开始索引 5077, 结束索引 5930, 长度 853
手势 8: 开始索引 5930, 结束索引 6793, 长度 863
手势 9: 开始索引 6793, 结束索引 7647, 长度 854
手势 10: 开始索引 7647, 结束索引 8501, 长度 854
手势 11: 开始索引 8501, 结束索引 9359, 长度 858
手势 12: 开始索引 9359, 结束索引 10211, 长度 852
手势 13: 开始索引 10211, 结束索引 11047, 长度 836
手势 14: 开始索引 11047, 结束索引 11871, 长度 824
手势 15: 开始索引 11871, 结束索引 12740, 长度 869
手势 16: 开始索引 12740, 结束索引 13583, 长度 843
手势 17: 开始索引 13583, 结束索引 14449, 长度 866
手势 18: 开始索引 14449, 结束索引 15302, 长度 853
手势 19: 开始索引 15302, 结束索引 16176, 长度 874
手势 20: 开始索引 16176, 结束索引 17031, 长度 855
手势 21: 开始索引 17031, 结束索引 17896, 长度 865
手势 22: 开始索引 17896, 结束索引 18733, 长度 837
手势 23: 开始索引 18733, 结束索引 19595, 长度 862
手势 24: 开始索引 19595, 结束索引 20443, 长度 848
手势 25: 开始索引 20443, 结束索引 

...


Processing sensor_data14.csv (14/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 848, 长度 848
手势 2: 开始索引 848, 结束索引 1699, 长度 851
手势 3: 开始索引 1699, 结束索引 2554, 长度 855
手势 4: 开始索引 2554, 结束索引 3422, 长度 868
手势 5: 开始索引 3422, 结束索引 4271, 长度 849
手势 6: 开始索引 4271, 结束索引 5133, 长度 862
手势 7: 开始索引 5133, 结束索引 5986, 长度 853
手势 8: 开始索引 5986, 结束索引 6839, 长度 853
手势 9: 开始索引 6839, 结束索引 7696, 长度 857
手势 10: 开始索引 7696, 结束索引 8552, 长度 856
手势 11: 开始索引 8552, 结束索引 9399, 长度 847
手势 12: 开始索引 9399, 结束索引 10248, 长度 849
手势 13: 开始索引 10248, 结束索引 11100, 长度 852
手势 14: 开始索引 11100, 结束索引 11951, 长度 851
手势 15: 开始索引 11951, 结束索引 12805, 长度 854
手势 16: 开始索引 12805, 结束索引 13654, 长度 849
手势 17: 开始索引 13654, 结束索引 14522, 长度 868
手势 18: 开始索引 14522, 结束索引 15350, 长度 828
手势 19: 开始索引 15350, 结束索引 16205, 长度 855
手势 20: 开始索引 16205, 结束索引 17056, 长度 851
手势 21: 开始索引 17056, 结束索引 17904, 长度 848
手势 22: 开始索引 17904, 结束索引 18756, 长度 852
手势 23: 开始索引 18756, 结束索引 19599, 长度 843
手势 24: 开始索引 19599, 结束索引 20453, 长度 854
手势 25: 开始索引 20453, 结束索

C:\Users\user\AppData\Local\Temp\ipykernel_26108\334003276.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.340880
1       -0.340880
2       -0.333884
3       -0.319892
4       -0.319892
           ...   
22165   -0.228942
22166   -0.214949
22167   -0.228942
22168   -0.235938
22169   -0.228942
Name: 272, Length: 22170, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\334003276.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.134902
1        0.119284
2        0.119284
3        0.127093
4        0.127093
           ...   
22165   -0.286769
22166   -0.278960
22167   -0.278960
22168   -0.271151
22169   -0.263343
Name: 328, Length: 22170, dtype: float64' has d


Processing sensor_data15.csv (15/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 862, 长度 862
手势 2: 开始索引 862, 结束索引 1706, 长度 844
手势 3: 开始索引 1706, 结束索引 2565, 长度 859
手势 4: 开始索引 2565, 结束索引 3400, 长度 835
手势 5: 开始索引 3400, 结束索引 4252, 长度 852
手势 6: 开始索引 4252, 结束索引 5110, 长度 858
手势 7: 开始索引 5110, 结束索引 5960, 长度 850
手势 8: 开始索引 5960, 结束索引 6794, 长度 834
手势 9: 开始索引 6794, 结束索引 7654, 长度 860
手势 10: 开始索引 7654, 结束索引 8500, 长度 846
手势 11: 开始索引 8500, 结束索引 9335, 长度 835
手势 12: 开始索引 9335, 结束索引 10203, 长度 868
手势 13: 开始索引 10203, 结束索引 11072, 长度 869
手势 14: 开始索引 11072, 结束索引 11909, 长度 837
手势 15: 开始索引 11909, 结束索引 12754, 长度 845
手势 16: 开始索引 12754, 结束索引 13606, 长度 852
手势 17: 开始索引 13606, 结束索引 14442, 长度 836
手势 18: 开始索引 14442, 结束索引 15283, 长度 841
手势 19: 开始索引 15283, 结束索引 16106, 长度 823
手势 20: 开始索引 16106, 结束索引 16959, 长度 853
手势 21: 开始索引 16959, 结束索引 17820, 长度 861
手势 22: 开始索引 17820, 结束索引 18675, 长度 855
手势 23: 开始索引 18675, 结束索引 19525, 长度 850
手势 24: 开始索引 19525, 结束索引 20384, 长度 859
手势 25: 开始索引 20384, 结束索

C:\Users\user\AppData\Local\Temp\ipykernel_26108\334003276.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.376132
1       -0.403871
2       -0.438546
3       -0.487090
4       -0.507895
           ...   
22056   -0.209694
22057   -0.209694
22058   -0.216629
22059   -0.209694
22060   -0.216629
Name: 268, Length: 22061, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\334003276.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.080763
1       -0.002722
2       -0.105473
3       -0.253178
4       -0.388039
           ...   
22056    0.022966
22057    0.022966
22058    0.016544
22059    0.022966
22060    0.022966
Name: 329, Length: 22061, dtype: float64' has d


Feature extraction complete! Shape: (390, 8, 15, 10)
Labels saved: (390,)


In [4]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\WXR\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



Loaded features from E:\MSC\Spring\AML\GestureLink\data\WXR\processed_data\feature_matrix.npy, shape: (390, 8, 15, 10)
Loaded labels from E:\MSC\Spring\AML\GestureLink\data\WXR\processed_data\labels.npy, shape: (390,)
Training set: X_train: (312, 8, 15, 10), y_train: (312,)
Testing set: X_test: (78, 8, 15, 10), y_test: (78,)


In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_data(data_folder, test_size=0.2, random_state=42):
    """
    加载 `feature_matrix.npy` 和 `labels.npy` 数据，并划分训练集和测试集。

    参数：
    - data_folder: 存放数据的文件夹路径
    - test_size: 测试集比例 (默认 20%)
    - random_state: 随机种子，保证可复现性

    返回：
    - X_train: 训练集特征，形状 (train_batches, num_windows, 15, num_channels)
    - X_test: 测试集特征，形状 (test_batches, num_windows, 15, num_channels)
    - y_train: 训练集标签，形状 (train_batches,)
    - y_test: 测试集标签，形状 (test_batches,)
    """
    # **加载数据**
    feature_path = os.path.join(data_folder, "feature_matrix.npy")
    label_path = os.path.join(data_folder, "labels.npy")

    if not os.path.exists(feature_path) or not os.path.exists(label_path):
        raise FileNotFoundError("特征文件或标签文件未找到，请检查路径！")

    X = np.load(feature_path)  # 形状 (num_batches, num_windows, 15, num_channels)
    y = np.load(label_path)  # 形状 (num_batches,)

    # **数据基本信息**
    print(f"Loaded features from {feature_path}, shape: {X.shape}")
    print(f"Loaded labels from {label_path}, shape: {y.shape}")

    # **划分训练集和测试集**
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

    # **打印数据划分信息**
    print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing set: X_test: {X_test.shape}, y_test: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# **使用示例**
data_folder = r"E:\MSC\Spring\AML\GestureLink\data\WXR\processed_data"
X_train, X_test, y_train, y_test = load_data(data_folder)



In [14]:
import numpy as np
import pandas as pd
import os
import re
import scipy.stats as stats
from scipy.signal import welch

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    处理 EMG 数据文件：
    - 通过 **时间戳间隔** 自动识别 26 个手势
    - **不足 5000 采样点的手势补零**
    - **确保最终 shape = (390, 49, 15, 10)，每个手势仅有一个标签**
    """
    fs = 1000  # 采样率 1000Hz
    window_size = 200  # 窗口大小 200 采样点（0.2s）
    step_size = 100  # 滑动步长 100 采样点（0.1s）
    time_gap_threshold = 500  # 手势分界时间间隔（单位：ms）

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # **按正确顺序读取 CSV 文件**
    csv_files = sorted(
        [f for f in os.listdir(input_folder) if f.startswith("sensor_data") and f.endswith(".csv")],
        key=lambda x: int(re.search(r"sensor_data(\d+)\.csv", x).group(1))
    )

    all_features = []
    all_labels = []

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"\nProcessing {file_name} ({file_idx + 1}/{len(csv_files)})")

        # **读取数据**
        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1  # 去掉时间列
        print(f"Detected {num_channels} channels (excluding time column).")

        # **归一化数据**
        data = raw_data.iloc[:, 1:]
        normalized_data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        # **获取时间戳**
        timestamps = raw_data.iloc[:, 0].astype(float).values  # 确保时间戳是 float 类型

        # **检测手势分界点（时间间隔 > 500ms）**
        segment_starts = [0]
        for i in range(1, len(timestamps)):
            if timestamps[i] - timestamps[i - 1] > time_gap_threshold:
                segment_starts.append(i)

        segment_starts = segment_starts[:26]  # 只保留前 26 个手势
        segment_ends = segment_starts[1:] + [len(raw_data)]  # 计算手势结束点

        print(f"\n[INFO] 识别到 {len(segment_starts)} 个手势")
        for i, (start, end) in enumerate(zip(segment_starts, segment_ends)):
            print(f"手势 {i+1}: 开始索引 {start}, 结束索引 {end}, 长度 {end - start}")

        # **切割手势数据**
        valid_segments = []
        for start, end in zip(segment_starts, segment_ends):
            segment = raw_data.iloc[start:end, 1:].values  # 提取特征列

            # **确保手势片段足够长，否则填充 0**
            if len(segment) < 900:
                # print(len(segment))
                pad_size = 900 - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            else:
                segment = segment[:900]  # 截断过长的手势

            valid_segments.append(segment)

        valid_segments = np.array(valid_segments)  # (26, 5000, num_channels)

        # **读取 `shuffle_order.xlsx` 以匹配手势标签**
        shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
        if file_idx >= len(shuffle_df):
            print(f"[WARNING] `shuffle_order.xlsx` 只有 {len(shuffle_df)} 行，但 `file_idx={file_idx}`，跳过该文件。")
            continue
        
        gesture_labels = shuffle_df.iloc[file_idx, :len(valid_segments)].values.tolist()  # 取前 `26` 个标签

        # **窗口化 & 特征提取**
        features_batches = []
        label_list = []
        for idx, seg in enumerate(valid_segments):
            num_samples = seg.shape[0]
            windows = [seg[j:j + window_size, :] for j in range(0, num_samples - window_size + 1, step_size)]
            
            batch_features = np.array([extract_features(window) for window in windows])  # (49, 15, num_channels)
            # features_batches.append(batch_features)
            features_batches.append(windows)

            # **仅保留 1 个手势的标签**
            label_list.append(gesture_labels[idx])  
        
        processed_segments = np.array(features_batches)  # (26, 49, 15, 10)
        print(processed_segments.shape)
        # **存储数据**
        all_features.append(processed_segments)
        all_labels.extend(label_list)  # **每个手势只对应 1 个标签**

    # **转换为 NumPy 数组**
    all_features = np.vstack(all_features)  # (390, 49, 15, 10)
    all_labels = np.array(all_labels)  # (390,)

    print(f"\nFeature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")

    # **保存数据**
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)


def extract_features(segment):
    """
    计算 EMG 和 IMU 的特征，返回 `float64` 类型
    """
    emg_signals = segment[:, :4].astype(np.float64)  # 4 个 EMG 通道
    imu_signals = segment[:, 4:].astype(np.float64)  # 6 个 IMU 通道

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ], dtype=np.float64)

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ], dtype=np.float64)

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1).astype(np.float64)  # (15, 10)


# **运行代码**
root = r"data\GZA"
input_folder = root
output_folder = os.path.join(root, "processed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")

process_emg_folder(input_folder, output_folder, shuffle_order_file)



Processing sensor_data1.csv (1/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 841, 长度 841
手势 2: 开始索引 841, 结束索引 1685, 长度 844
手势 3: 开始索引 1685, 结束索引 2517, 长度 832
手势 4: 开始索引 2517, 结束索引 3352, 长度 835
手势 5: 开始索引 3352, 结束索引 4196, 长度 844
手势 6: 开始索引 4196, 结束索引 5032, 长度 836
手势 7: 开始索引 5032, 结束索引 5882, 长度 850
手势 8: 开始索引 5882, 结束索引 6728, 长度 846
手势 9: 开始索引 6728, 结束索引 7578, 长度 850
手势 10: 开始索引 7578, 结束索引 8421, 长度 843
手势 11: 开始索引 8421, 结束索引 9256, 长度 835
手势 12: 开始索引 9256, 结束索引 10091, 长度 835
手势 13: 开始索引 10091, 结束索引 10943, 长度 852
手势 14: 开始索引 10943, 结束索引 11780, 长度 837
手势 15: 开始索引 11780, 结束索引 12628, 长度 848
手势 16: 开始索引 12628, 结束索引 13478, 长度 850
手势 17: 开始索引 13478, 结束索引 14330, 长度 852
手势 18: 开始索引 14330, 结束索引 15176, 长度 846
手势 19: 开始索引 15176, 结束索引 16016, 长度 840
手势 20: 开始索引 16016, 结束索引 16889, 长度 873
手势 21: 开始索引 16889, 结束索引 17739, 长度 850
手势 22: 开始索引 17739, 结束索引 18583, 长度 844
手势 23: 开始索引 18583, 结束索引 19427, 长度 844
手势 24: 开始索引 19427, 结束索引 20272, 长度 845
手势 25: 开始索引 20272, 结束索引 

C:\Users\user\AppData\Local\Temp\ipykernel_26108\394251214.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.129107
1       -0.098329
2       -0.075245
3       -0.106023
4       -0.106023
           ...   
21951   -0.221441
21952   -0.229135
21953   -0.267608
21954   -0.298386
21955   -0.290691
Name: 297, Length: 21956, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\394251214.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.046483
1        0.031909
2        0.031909
3        0.031909
4        0.031909
           ...   
21951    0.061056
21952    0.046483
21953    0.061056
21954    0.046483
21955    0.017336
Name: 316, Length: 21956, dtype: float64' has d

(26, 8, 200, 10)

Processing sensor_data2.csv (2/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 841, 长度 841
手势 2: 开始索引 841, 结束索引 1679, 长度 838
手势 3: 开始索引 1679, 结束索引 2519, 长度 840
手势 4: 开始索引 2519, 结束索引 3357, 长度 838
手势 5: 开始索引 3357, 结束索引 4201, 长度 844
手势 6: 开始索引 4201, 结束索引 5012, 长度 811
手势 7: 开始索引 5012, 结束索引 5859, 长度 847
手势 8: 开始索引 5859, 结束索引 6701, 长度 842
手势 9: 开始索引 6701, 结束索引 7535, 长度 834
手势 10: 开始索引 7535, 结束索引 8390, 长度 855
手势 11: 开始索引 8390, 结束索引 9231, 长度 841
手势 12: 开始索引 9231, 结束索引 10066, 长度 835
手势 13: 开始索引 10066, 结束索引 10892, 长度 826
手势 14: 开始索引 10892, 结束索引 11742, 长度 850
手势 15: 开始索引 11742, 结束索引 12570, 长度 828
手势 16: 开始索引 12570, 结束索引 13420, 长度 850
手势 17: 开始索引 13420, 结束索引 14278, 长度 858
手势 18: 开始索引 14278, 结束索引 15099, 长度 821
手势 19: 开始索引 15099, 结束索引 15933, 长度 834
手势 20: 开始索引 15933, 结束索引 16781, 长度 848
手势 21: 开始索引 16781, 结束索引 17638, 长度 857
手势 22: 开始索引 17638, 结束索引 18471, 长度 833
手势 23: 开始索引 18471, 结束索引 19317, 长度 846
手势 24: 开始索引 19317, 结束索引 20165, 长度 848
手势 25: 

C:\Users\user\AppData\Local\Temp\ipykernel_26108\394251214.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.374197
1        0.397380
2        0.339422
3        0.339422
4        0.374197
           ...   
21803   -0.135838
21804   -0.031513
21805   -0.101063
21806   -0.147429
21807   -0.031513
Name: 338, Length: 21808, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\394251214.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.690661
1        0.690661
2        0.713441
3        0.713441
4        0.713441
           ...   
21803   -0.038304
21804   -0.061084
21805   -0.038304
21806   -0.038304
21807   -0.038304
Name: 342, Length: 21808, dtype: float64' has d

(26, 8, 200, 10)

Processing sensor_data3.csv (3/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 841, 长度 841
手势 2: 开始索引 841, 结束索引 1675, 长度 834
手势 3: 开始索引 1675, 结束索引 2519, 长度 844
手势 4: 开始索引 2519, 结束索引 3380, 长度 861
手势 5: 开始索引 3380, 结束索引 4232, 长度 852
手势 6: 开始索引 4232, 结束索引 5079, 长度 847
手势 7: 开始索引 5079, 结束索引 5911, 长度 832
手势 8: 开始索引 5911, 结束索引 6755, 长度 844
手势 9: 开始索引 6755, 结束索引 7588, 长度 833
手势 10: 开始索引 7588, 结束索引 8440, 长度 852
手势 11: 开始索引 8440, 结束索引 9276, 长度 836
手势 12: 开始索引 9276, 结束索引 10098, 长度 822
手势 13: 开始索引 10098, 结束索引 10938, 长度 840
手势 14: 开始索引 10938, 结束索引 11788, 长度 850
手势 15: 开始索引 11788, 结束索引 12640, 长度 852
手势 16: 开始索引 12640, 结束索引 13451, 长度 811
手势 17: 开始索引 13451, 结束索引 14301, 长度 850
手势 18: 开始索引 14301, 结束索引 15157, 长度 856
手势 19: 开始索引 15157, 结束索引 16005, 长度 848
手势 20: 开始索引 16005, 结束索引 16851, 长度 846
手势 21: 开始索引 16851, 结束索引 17696, 长度 845
手势 22: 开始索引 17696, 结束索引 18547, 长度 851
手势 23: 开始索引 18547, 结束索引 19382, 长度 835
手势 24: 开始索引 19382, 结束索引 20227, 长度 845
手势 25: 

...

(26, 8, 200, 10)

Processing sensor_data14.csv (14/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 830, 长度 830
手势 2: 开始索引 830, 结束索引 1669, 长度 839
手势 3: 开始索引 1669, 结束索引 2529, 长度 860
手势 4: 开始索引 2529, 结束索引 3369, 长度 840
手势 5: 开始索引 3369, 结束索引 4204, 长度 835
手势 6: 开始索引 4204, 结束索引 5052, 长度 848
手势 7: 开始索引 5052, 结束索引 5882, 长度 830
手势 8: 开始索引 5882, 结束索引 6716, 长度 834
手势 9: 开始索引 6716, 结束索引 7550, 长度 834
手势 10: 开始索引 7550, 结束索引 8392, 长度 842
手势 11: 开始索引 8392, 结束索引 9233, 长度 841
手势 12: 开始索引 9233, 结束索引 10065, 长度 832
手势 13: 开始索引 10065, 结束索引 10891, 长度 826
手势 14: 开始索引 10891, 结束索引 11727, 长度 836
手势 15: 开始索引 11727, 结束索引 12550, 长度 823
手势 16: 开始索引 12550, 结束索引 13383, 长度 833
手势 17: 开始索引 13383, 结束索引 14226, 长度 843
手势 18: 开始索引 14226, 结束索引 15072, 长度 846
手势 19: 开始索引 15072, 结束索引 15910, 长度 838
手势 20: 开始索引 15910, 结束索引 16755, 长度 845
手势 21: 开始索引 16755, 结束索引 17589, 长度 834
手势 22: 开始索引 17589, 结束索引 18428, 长度 839
手势 23: 开始索引 18428, 结束索引 19270, 长度 842
手势 24: 开始索引 19270, 结束索引 20118, 长度 848
手势 25

C:\Users\user\AppData\Local\Temp\ipykernel_26108\394251214.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.232971
1       -0.232971
2       -0.211098
3       -0.254844
4       -0.276717
           ...   
21804   -0.112669
21805   -0.156415
21806   -0.145479
21807   -0.112669
21808   -0.145479
Name: 307, Length: 21809, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\394251214.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.102345
1       -0.102345
2       -0.073627
3       -0.087986
4       -0.102345
           ...   
21804   -0.044908
21805   -0.044908
21806   -0.044908
21807   -0.030549
21808   -0.030549
Name: 306, Length: 21809, dtype: float64' has d

(26, 8, 200, 10)

Processing sensor_data15.csv (15/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 845, 长度 845
手势 2: 开始索引 845, 结束索引 1690, 长度 845
手势 3: 开始索引 1690, 结束索引 2544, 长度 854
手势 4: 开始索引 2544, 结束索引 3392, 长度 848
手势 5: 开始索引 3392, 结束索引 4240, 长度 848
手势 6: 开始索引 4240, 结束索引 5089, 长度 849
手势 7: 开始索引 5089, 结束索引 5930, 长度 841
手势 8: 开始索引 5930, 结束索引 6767, 长度 837
手势 9: 开始索引 6767, 结束索引 7623, 长度 856
手势 10: 开始索引 7623, 结束索引 8466, 长度 843
手势 11: 开始索引 8466, 结束索引 9309, 长度 843
手势 12: 开始索引 9309, 结束索引 10162, 长度 853
手势 13: 开始索引 10162, 结束索引 10994, 长度 832
手势 14: 开始索引 10994, 结束索引 11822, 长度 828
手势 15: 开始索引 11822, 结束索引 12668, 长度 846
手势 16: 开始索引 12668, 结束索引 13543, 长度 875
手势 17: 开始索引 13543, 结束索引 14386, 长度 843
手势 18: 开始索引 14386, 结束索引 15238, 长度 852
手势 19: 开始索引 15238, 结束索引 16088, 长度 850
手势 20: 开始索引 16088, 结束索引 16935, 长度 847
手势 21: 开始索引 16935, 结束索引 17778, 长度 843
手势 22: 开始索引 17778, 结束索引 18625, 长度 847
手势 23: 开始索引 18625, 结束索引 19474, 长度 849
手势 24: 开始索引 19474, 结束索引 20328, 长度 854
手势 25

C:\Users\user\AppData\Local\Temp\ipykernel_26108\394251214.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.185436
1        0.149263
2        0.185436
3        0.209552
4        0.161321
           ...   
21983    0.004569
21984    0.040743
21985   -0.031604
21986   -0.019546
21987    0.028685
Name: 336, Length: 21988, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\394251214.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.068719
1       -0.098047
2       -0.083383
3       -0.068719
4       -0.083383
           ...   
21983    0.121918
21984    0.136582
21985    0.092589
21986    0.077925
21987    0.107253
Name: 307, Length: 21988, dtype: float64' has d

(26, 8, 200, 10)

Feature extraction complete! Shape: (390, 8, 200, 10)
Labels saved: (390,)


In [7]:
import numpy as np
import pandas as pd
import os
import re
import scipy.stats as stats
from scipy.signal import welch

def process_emg_folder(input_folder, output_folder, shuffle_order_file):
    """
    处理 EMG 数据文件：
    - 通过 **时间戳间隔** 自动识别 26 个手势
    - **不足 5000 采样点的手势补零**
    - **确保最终 shape = (390, 49, 15, 10)，每个手势仅有一个标签**
    """
    fs = 1000  # 采样率 1000Hz
    window_size = 200  # 窗口大小 200 采样点（0.2s）
    step_size = 100  # 滑动步长 100 采样点（0.1s）
    time_gap_threshold = 500  # 手势分界时间间隔（单位：ms）

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # **按正确顺序读取 CSV 文件**
    csv_files = sorted(
        [f for f in os.listdir(input_folder) if f.startswith("sensor_data") and f.endswith(".csv")],
        key=lambda x: int(re.search(r"sensor_data(\d+)\.csv", x).group(1))
    )

    all_features = []
    all_labels = []

    for file_idx, file_name in enumerate(csv_files):
        file_path = os.path.join(input_folder, file_name)
        print(f"\nProcessing {file_name} ({file_idx + 1}/{len(csv_files)})")

        # **读取数据**
        raw_data = pd.read_csv(file_path)
        num_channels = raw_data.shape[1] - 1  # 去掉时间列
        print(f"Detected {num_channels} channels (excluding time column).")

        # **归一化数据**
        data = raw_data.iloc[:, 1:]
        normalized_data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-10)
        raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)

        # **获取时间戳**
        timestamps = raw_data.iloc[:, 0].astype(float).values  # 确保时间戳是 float 类型

        # **检测手势分界点（时间间隔 > 500ms）**
        segment_starts = [0]
        for i in range(1, len(timestamps)):
            if timestamps[i] - timestamps[i - 1] > time_gap_threshold:
                segment_starts.append(i)

        segment_starts = segment_starts[:26]  # 只保留前 26 个手势
        segment_ends = segment_starts[1:] + [len(raw_data)]  # 计算手势结束点

        print(f"\n[INFO] 识别到 {len(segment_starts)} 个手势")
        for i, (start, end) in enumerate(zip(segment_starts, segment_ends)):
            print(f"手势 {i+1}: 开始索引 {start}, 结束索引 {end}, 长度 {end - start}")

        # **切割手势数据**
        valid_segments = []
        for start, end in zip(segment_starts, segment_ends):
            segment = raw_data.iloc[start:end, 1:].values  # 提取特征列

            # **确保手势片段足够长，否则填充 0**
            if len(segment) < 900:
                # print(len(segment))
                pad_size = 900 - len(segment)
                segment = np.pad(segment, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            else:
                segment = segment[:900]  # 截断过长的手势

            valid_segments.append(segment)

        valid_segments = np.array(valid_segments)  # (26, 5000, num_channels)

        # **读取 `shuffle_order.xlsx` 以匹配手势标签**
        shuffle_df = pd.read_excel(shuffle_order_file, engine="openpyxl")
        if file_idx >= len(shuffle_df):
            print(f"[WARNING] `shuffle_order.xlsx` 只有 {len(shuffle_df)} 行，但 `file_idx={file_idx}`，跳过该文件。")
            continue
        
        gesture_labels = shuffle_df.iloc[file_idx, :len(valid_segments)].values.tolist()  # 取前 `26` 个标签

        # **窗口化 & 特征提取**
        features_batches = []
        label_list = []
        for idx, seg in enumerate(valid_segments):
            num_samples = seg.shape[0]
            windows = [seg[j:j + window_size, :] for j in range(0, num_samples - window_size + 1, step_size)]
            
            batch_features = np.array([extract_features(window) for window in windows])  # (49, 15, num_channels)
            features_batches.append(batch_features)

            # **仅保留 1 个手势的标签**
            label_list.append(gesture_labels[idx])  

        processed_segments = np.array(features_batches)  # (26, 49, 15, 10)

        # **存储数据**
        all_features.append(processed_segments)
        all_labels.extend(label_list)  # **每个手势只对应 1 个标签**

    # **转换为 NumPy 数组**
    all_features = np.vstack(all_features)  # (390, 49, 15, 10)
    all_labels = np.array(all_labels)  # (390,)

    print(f"\nFeature extraction complete! Shape: {all_features.shape}")
    print(f"Labels saved: {all_labels.shape}")

    # **保存数据**
    np.save(os.path.join(output_folder, "feature_matrix.npy"), all_features)
    np.save(os.path.join(output_folder, "labels.npy"), all_labels)


def extract_features(segment):
    """
    计算 EMG 和 IMU 的特征，返回 `float64` 类型
    """
    emg_signals = segment[:, :4].astype(np.float64)  # 4 个 EMG 通道
    imu_signals = segment[:, 4:].astype(np.float64)  # 6 个 IMU 通道

    def compute_emg_features(signal):
        return np.array([
            np.var(signal), np.mean(np.abs(signal)), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.mean(np.abs(np.diff(signal))), np.max(signal),
            np.min(signal), np.sum(np.diff(signal) > 0), np.sum(np.diff(np.sign(signal)) != 0),
            stats.kurtosis(signal), stats.skew(signal), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1])
        ], dtype=np.float64)

    def compute_imu_features(signal):
        return np.array([
            np.var(signal), np.mean(signal), np.sqrt(np.mean(signal**2)),
            np.std(signal), np.max(signal), np.min(signal),
            stats.kurtosis(signal), stats.skew(signal),
            np.mean(np.abs(np.diff(signal))), np.sum(np.abs(np.diff(signal))),
            np.sum(signal ** 2), np.log10(np.mean(signal**2) + 1e-10),
            np.mean(welch(signal, fs=1000, nperseg=200)[1]), np.median(signal), np.ptp(signal)
        ], dtype=np.float64)

    emg_features = np.array([compute_emg_features(emg_signals[:, i]) for i in range(4)]).T  # (15, 4)
    imu_features = np.array([compute_imu_features(imu_signals[:, i]) for i in range(6)]).T  # (15, 6)

    return np.concatenate((emg_features, imu_features), axis=1).astype(np.float64)  # (15, 10)


# **运行代码**
root = r"data\FZH"
input_folder = root
output_folder = os.path.join(root, "processed_data")
shuffle_order_file = os.path.join(root, "shuffle_order.xlsx")

process_emg_folder(input_folder, output_folder, shuffle_order_file)



Processing sensor_data1.csv (1/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 846, 长度 846
手势 2: 开始索引 846, 结束索引 1702, 长度 856
手势 3: 开始索引 1702, 结束索引 2586, 长度 884
手势 4: 开始索引 2586, 结束索引 3429, 长度 843
手势 5: 开始索引 3429, 结束索引 4276, 长度 847
手势 6: 开始索引 4276, 结束索引 5122, 长度 846
手势 7: 开始索引 5122, 结束索引 5975, 长度 853
手势 8: 开始索引 5975, 结束索引 6825, 长度 850
手势 9: 开始索引 6825, 结束索引 7689, 长度 864
手势 10: 开始索引 7689, 结束索引 8534, 长度 845
手势 11: 开始索引 8534, 结束索引 9388, 长度 854
手势 12: 开始索引 9388, 结束索引 10262, 长度 874
手势 13: 开始索引 10262, 结束索引 11140, 长度 878
手势 14: 开始索引 11140, 结束索引 12013, 长度 873
手势 15: 开始索引 12013, 结束索引 12854, 长度 841
手势 16: 开始索引 12854, 结束索引 13708, 长度 854
手势 17: 开始索引 13708, 结束索引 14561, 长度 853
手势 18: 开始索引 14561, 结束索引 15408, 长度 847
手势 19: 开始索引 15408, 结束索引 16267, 长度 859
手势 20: 开始索引 16267, 结束索引 17149, 长度 882
手势 21: 开始索引 17149, 结束索引 18012, 长度 863
手势 22: 开始索引 18012, 结束索引 18841, 长度 829
手势 23: 开始索引 18841, 结束索引 19697, 长度 856
手势 24: 开始索引 19697, 结束索引 20584, 长度 887
手势 25: 开始索引 20584, 结束索引 

C:\Users\user\AppData\Local\Temp\ipykernel_26108\188611202.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.033795
1       -0.033795
2       -0.033795
3       -0.033795
4       -0.033795
           ...   
22275   -0.033795
22276   -0.033795
22277   -0.033795
22278   -0.033795
22279   -0.033795
Name: 303, Length: 22280, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\188611202.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.132256
1       -0.132256
2       -0.114027
3       -0.123142
4       -0.132256
           ...   
22275    1.079998
22276    1.079998
22277    1.061769
22278    1.061769
22279    1.070883
Name: 293, Length: 22280, dtype: float64' has d


Processing sensor_data2.csv (2/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 878, 长度 878
手势 2: 开始索引 878, 结束索引 1747, 长度 869
手势 3: 开始索引 1747, 结束索引 2596, 长度 849
手势 4: 开始索引 2596, 结束索引 3432, 长度 836
手势 5: 开始索引 3432, 结束索引 4285, 长度 853
手势 6: 开始索引 4285, 结束索引 5151, 长度 866
手势 7: 开始索引 5151, 结束索引 6017, 长度 866
手势 8: 开始索引 6017, 结束索引 6867, 长度 850
手势 9: 开始索引 6867, 结束索引 7742, 长度 875
手势 10: 开始索引 7742, 结束索引 8594, 长度 852
手势 11: 开始索引 8594, 结束索引 9432, 长度 838
手势 12: 开始索引 9432, 结束索引 10280, 长度 848
手势 13: 开始索引 10280, 结束索引 11110, 长度 830
手势 14: 开始索引 11110, 结束索引 11967, 长度 857
手势 15: 开始索引 11967, 结束索引 12831, 长度 864
手势 16: 开始索引 12831, 结束索引 13663, 长度 832
手势 17: 开始索引 13663, 结束索引 14519, 长度 856
手势 18: 开始索引 14519, 结束索引 15353, 长度 834
手势 19: 开始索引 15353, 结束索引 16200, 长度 847
手势 20: 开始索引 16200, 结束索引 17021, 长度 821
手势 21: 开始索引 17021, 结束索引 17863, 长度 842
手势 22: 开始索引 17863, 结束索引 18714, 长度 851
手势 23: 开始索引 18714, 结束索引 19558, 长度 844
手势 24: 开始索引 19558, 结束索引 20429, 长度 871
手势 25: 开始索引 20429, 结束索引 

C:\Users\user\AppData\Local\Temp\ipykernel_26108\188611202.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        2.477226
1        1.176720
2       -0.123785
3        1.176720
4        1.176720
           ...   
22121   -0.123785
22122   -0.123785
22123   -0.123785
22124   -0.123785
22125   -0.123785
Name: 306, Length: 22126, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\188611202.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.093122
1        0.103353
2        0.021503
3       -0.019423
4        0.062428
           ...   
22121    0.072659
22122    0.093122
22123    0.082891
22124    0.103353
22125    0.134047
Name: 313, Length: 22126, dtype: float64' has d


Processing sensor_data3.csv (3/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 845, 长度 845
手势 2: 开始索引 845, 结束索引 1678, 长度 833
手势 3: 开始索引 1678, 结束索引 2528, 长度 850
手势 4: 开始索引 2528, 结束索引 3376, 长度 848
手势 5: 开始索引 3376, 结束索引 4232, 长度 856
手势 6: 开始索引 4232, 结束索引 5100, 长度 868
手势 7: 开始索引 5100, 结束索引 5977, 长度 877
手势 8: 开始索引 5977, 结束索引 6860, 长度 883
手势 9: 开始索引 6860, 结束索引 7735, 长度 875
手势 10: 开始索引 7735, 结束索引 8585, 长度 850
手势 11: 开始索引 8585, 结束索引 9446, 长度 861
手势 12: 开始索引 9446, 结束索引 10303, 长度 857
手势 13: 开始索引 10303, 结束索引 11148, 长度 845
手势 14: 开始索引 11148, 结束索引 12008, 长度 860
手势 15: 开始索引 12008, 结束索引 12866, 长度 858
手势 16: 开始索引 12866, 结束索引 13731, 长度 865
手势 17: 开始索引 13731, 结束索引 14592, 长度 861
手势 18: 开始索引 14592, 结束索引 15441, 长度 849
手势 19: 开始索引 15441, 结束索引 16313, 长度 872
手势 20: 开始索引 16313, 结束索引 17175, 长度 862
手势 21: 开始索引 17175, 结束索引 18017, 长度 842
手势 22: 开始索引 18017, 结束索引 18878, 长度 861
手势 23: 开始索引 18878, 结束索引 19729, 长度 851
手势 24: 开始索引 19729, 结束索引 20607, 长度 878
手势 25: 开始索引 20607, 结束索引 

...


Processing sensor_data14.csv (14/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 846, 长度 846
手势 2: 开始索引 846, 结束索引 1697, 长度 851
手势 3: 开始索引 1697, 结束索引 2562, 长度 865
手势 4: 开始索引 2562, 结束索引 3412, 长度 850
手势 5: 开始索引 3412, 结束索引 4261, 长度 849
手势 6: 开始索引 4261, 结束索引 5124, 长度 863
手势 7: 开始索引 5124, 结束索引 5980, 长度 856
手势 8: 开始索引 5980, 结束索引 6841, 长度 861
手势 9: 开始索引 6841, 结束索引 7706, 长度 865
手势 10: 开始索引 7706, 结束索引 8559, 长度 853
手势 11: 开始索引 8559, 结束索引 9414, 长度 855
手势 12: 开始索引 9414, 结束索引 10266, 长度 852
手势 13: 开始索引 10266, 结束索引 11126, 长度 860
手势 14: 开始索引 11126, 结束索引 11998, 长度 872
手势 15: 开始索引 11998, 结束索引 12866, 长度 868
手势 16: 开始索引 12866, 结束索引 13719, 长度 853
手势 17: 开始索引 13719, 结束索引 14566, 长度 847
手势 18: 开始索引 14566, 结束索引 15405, 长度 839
手势 19: 开始索引 15405, 结束索引 16260, 长度 855
手势 20: 开始索引 16260, 结束索引 17111, 长度 851
手势 21: 开始索引 17111, 结束索引 17976, 长度 865
手势 22: 开始索引 17976, 结束索引 18837, 长度 861
手势 23: 开始索引 18837, 结束索引 19704, 长度 867
手势 24: 开始索引 19704, 结束索引 20583, 长度 879
手势 25: 开始索引 20583, 结束索

C:\Users\user\AppData\Local\Temp\ipykernel_26108\188611202.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.660634
1       -1.962066
2       -0.660634
3       -0.660634
4        0.640797
           ...   
22301   -0.660634
22302   -0.660634
22303    0.640797
22304   -0.660634
22305   -0.660634
Name: 304, Length: 22306, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\188611202.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.352143
1        0.391314
2        0.430486
3        0.469657
4        0.456600
           ...   
22301    0.025714
22302   -0.000400
22303   -0.026515
22304   -0.000400
22305   -0.000400
Name: 329, Length: 22306, dtype: float64' has d


Processing sensor_data15.csv (15/15)
Detected 10 channels (excluding time column).

[INFO] 识别到 26 个手势
手势 1: 开始索引 0, 结束索引 851, 长度 851
手势 2: 开始索引 851, 结束索引 1720, 长度 869
手势 3: 开始索引 1720, 结束索引 2566, 长度 846
手势 4: 开始索引 2566, 结束索引 3430, 长度 864
手势 5: 开始索引 3430, 结束索引 4274, 长度 844
手势 6: 开始索引 4274, 结束索引 5140, 长度 866
手势 7: 开始索引 5140, 结束索引 5997, 长度 857
手势 8: 开始索引 5997, 结束索引 6858, 长度 861
手势 9: 开始索引 6858, 结束索引 7731, 长度 873
手势 10: 开始索引 7731, 结束索引 8593, 长度 862
手势 11: 开始索引 8593, 结束索引 9450, 长度 857
手势 12: 开始索引 9450, 结束索引 10318, 长度 868
手势 13: 开始索引 10318, 结束索引 11172, 长度 854
手势 14: 开始索引 11172, 结束索引 12046, 长度 874
手势 15: 开始索引 12046, 结束索引 12926, 长度 880
手势 16: 开始索引 12926, 结束索引 13774, 长度 848
手势 17: 开始索引 13774, 结束索引 14619, 长度 845
手势 18: 开始索引 14619, 结束索引 15498, 长度 879
手势 19: 开始索引 15498, 结束索引 16391, 长度 893
手势 20: 开始索引 16391, 结束索引 17226, 长度 835
手势 21: 开始索引 17226, 结束索引 18101, 长度 875
手势 22: 开始索引 18101, 结束索引 18949, 长度 848
手势 23: 开始索引 18949, 结束索引 19825, 长度 876
手势 24: 开始索引 19825, 结束索引 20697, 长度 872
手势 25: 开始索引 20697, 结束索

C:\Users\user\AppData\Local\Temp\ipykernel_26108\188611202.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        0.709376
1        0.709376
2        0.709376
3        0.709376
4       -1.018934
           ...   
22456   -1.018934
22457    0.709376
22458    0.709376
22459   -1.018934
22460    0.709376
Name: 305, Length: 22461, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  raw_data.iloc[:, 1:] = normalized_data.astype(np.float64)
C:\Users\user\AppData\Local\Temp\ipykernel_26108\188611202.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       -0.466781
1       -0.429730
2       -0.367977
3       -0.330925
4       -0.306224
           ...   
22456   -0.022162
22457   -0.046863
22458   -0.034513
22459   -0.034513
22460   -0.046863
Name: 269, Length: 22461, dtype: float64' has d


Feature extraction complete! Shape: (390, 8, 15, 10)
Labels saved: (390,)
